### Mutational data loading and preprocessing

In [1]:
# import python packages
import shap
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn import metrics, calibration
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score  
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
# allow multiple outputs of a cell to be displayed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Load mutation data

In [3]:
import os
os.chdir('/home/projects/aafek/nogal/noga_project/AI_hub/Repair-mutation project/Data')

"merged_genomic_features_TDG_results.csv" contains all C to T mutations reported in dbSNP for chromosome 1.

In [4]:
#relevant cols to load
columns_to_load = ['sequence_context', 'seqnames', 'start', 'CAF', 'PhyloP_score', 'is_CpG', 'gene_type', 'protein_id', 
                   'ccre_type', 'log.Intensity..RU.']

In [5]:
df = pd.read_csv('merged_genomic_features_phylop241_TDG_results.csv', usecols=columns_to_load)

# Filter rows without reported PhyloP_score and CAF (which will be the labels)
filtered_df = df.dropna(subset=['PhyloP_score', 'CAF'])

# Determine the minimum number of samples to take from each sequence context
min_samples_per_context = 10  # Adjust as needed

# Group by 'sequence_context' and sample equal rows from each context
sampled_df = filtered_df.groupby('sequence_context', group_keys=False).apply(
    lambda x: x.sample(n=min(min_samples_per_context, len(x)), random_state=42))

# Check the result to make sure that every context is represented equally:
print(sampled_df['sequence_context'].value_counts())

sequence_context
AACAAAA    10
GGCGGTA    10
GGCGCTT    10
GGCGGAA    10
GGCGGAC    10
           ..
CGCCGAA     3
CGCTCGT     3
GTCCGCG     3
CGCTACG     1
CGCAACG     1
Name: count, Length: 4096, dtype: int64


In [6]:
len(sampled_df['sequence_context'].value_counts()) # to see how many different contexts I have (should be 4096)

4096

In [7]:
# take a look at the raw data
sampled_df

,sequence_context,seqnames,start,CAF,PhyloP_score,is_CpG,gene_type,protein_id,ccre_type,log.Intensity..RU.
7636,AACAAAA,1,99267131,"0.992,0.007987",-0.055,0,protein_coding,ENSP00000359204.4,NaN,7.090493
8915,AACAAAA,1,161294824,"0.9998,0.0001997",-1.399,0,NaN,NaN,NaN,7.090493
908,AACAAAA,1,169903374,"0.9996,0.0003994",-1.708,0,NaN,NaN,NaN,7.090493
5248,AACAAAA,1,82064640,"0.9998,0.0001997",-0.639,0,NaN,NaN,NaN,7.090493
4127,AACAAAA,1,153830104,"0.9996,0.0003994",-1.016,0,protein_coding,ENSP00000490635.1,NaN,7.090493
...,...,...,...,...,...,...,...,...,...,...
7582302,TTCTTTT,1,156911602,"0.9792,0.02077",-0.868,0,protein_coding,NaN,NaN,6.576470
7578604,TTCTTTT,1,26226773,"0.5325,0.4675",-1.496,0,transcribed_unprocessed_pseudogene,NaN,NaN,6.576470
7591827,TTCTTTT,1,229773166,"0.9998,0.0001997",0.243,0,NaN,NaN,NaN,6.576470
7579876,TTCTTTT,1,106091814,"0.7051,0.2949",-1.192,0,lncRNA,NaN,NaN,6.576470


### Extract MAF

In [8]:
# Extract Minor Allele Frequency (MAF) from the CAF column
# The CAF column contains comma-separated values. We'll split it and take the minimum non-zero value.
def calculate_maf(caf):
    if pd.isna(caf):
        return None
    frequencies = [float(freq) for freq in caf.split(',') if freq != '']
    return min(frequencies) if frequencies else None

sampled_df['MAF'] = sampled_df['CAF'].apply(calculate_maf)

Make NAs be "0" for regions where genes/regulatory regions are not defined

In [9]:
# Fill missing values in specific columns with 0
columns_to_fill = ['gene_type', 'protein_id', 'ccre_type']
sampled_df[columns_to_fill] = sampled_df[columns_to_fill].fillna("0")

In [10]:
# Remove contexts that have below 100 samples
# Add a column for counts per sequence_context
sampled_df['label_counts'] = sampled_df['sequence_context'].map(sampled_df['sequence_context'].value_counts())

# Filter contexts with 100 or more occurrences
sampled_df_100 = sampled_df[sampled_df['label_counts'] > 9]

In [11]:
# Check the modified data
sampled_df.shape, sampled_df_100.shape

((40845, 12), (40670, 12))

Transform categoric to numeric for XGBoost

In [12]:
#generate a copy of the table
sampled_df_100_transformed = sampled_df_100.copy()

In [13]:
categorical_columns = ['gene_type', 'ccre_type']  # Add categorical columns here

# Apply one hot encoding on categoric cols:
sampled_df_100_transformed = pd.get_dummies(sampled_df_100_transformed, columns=categorical_columns, drop_first=True)
sampled_df_100_transformed

,sequence_context,seqnames,start,CAF,PhyloP_score,is_CpG,protein_id,log.Intensity..RU.,MAF,label_counts,...,gene_type_unprocessed_pseudogene,"ccre_type_CTCF-only,CTCF-bound",ccre_type_DNase-H3K4me3,"ccre_type_DNase-H3K4me3,CTCF-bound",ccre_type_PLS,"ccre_type_PLS,CTCF-bound",ccre_type_dELS,"ccre_type_dELS,CTCF-bound",ccre_type_pELS,"ccre_type_pELS,CTCF-bound"
7636,AACAAAA,1,99267131,"0.992,0.007987",-0.055,0,ENSP00000359204.4,7.090493,0.007987,10,...,False,False,False,False,False,False,False,False,False,False
8915,AACAAAA,1,161294824,"0.9998,0.0001997",-1.399,0,0,7.090493,0.000200,10,...,False,False,False,False,False,False,False,False,False,False
908,AACAAAA,1,169903374,"0.9996,0.0003994",-1.708,0,0,7.090493,0.000399,10,...,False,False,False,False,False,False,False,False,False,False
5248,AACAAAA,1,82064640,"0.9998,0.0001997",-0.639,0,0,7.090493,0.000200,10,...,False,False,False,False,False,False,False,False,False,False
4127,AACAAAA,1,153830104,"0.9996,0.0003994",-1.016,0,ENSP00000490635.1,7.090493,0.000399,10,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582302,TTCTTTT,1,156911602,"0.9792,0.02077",-0.868,0,0,6.576470,0.020770,10,...,False,False,False,False,False,False,False,False,False,False
7578604,TTCTTTT,1,26226773,"0.5325,0.4675",-1.496,0,0,6.576470,0.467500,10,...,False,False,False,False,False,False,False,False,False,False
7591827,TTCTTTT,1,229773166,"0.9998,0.0001997",0.243,0,0,6.576470,0.000200,10,...,False,False,False,False,False,False,False,False,False,False
7579876,TTCTTTT,1,106091814,"0.7051,0.2949",-1.192,0,0,6.576470,0.294900,10,...,False,False,False,False,False,False,False,False,False,False


In [14]:
# transform all protein coding genes to "1":
sampled_df_100_transformed.loc[:, 'is_protein_coding'] = sampled_df_100_transformed['protein_id'].apply(lambda x: 1 if isinstance(x, str) and len(x) > 0 else 0)

In [15]:
# transform MAF to log(MAF)
epsilon = 1e-7
sampled_df_100_transformed['MAF_log'] = np.log(sampled_df_100_transformed['MAF'] + epsilon)
# rename experimental col:
sampled_df_100_transformed.rename(columns={"log.Intensity..RU.": "experimental_ln_FI"}, inplace=True)
sampled_df_100_transformed

,sequence_context,seqnames,start,CAF,PhyloP_score,is_CpG,protein_id,experimental_ln_FI,MAF,label_counts,...,ccre_type_DNase-H3K4me3,"ccre_type_DNase-H3K4me3,CTCF-bound",ccre_type_PLS,"ccre_type_PLS,CTCF-bound",ccre_type_dELS,"ccre_type_dELS,CTCF-bound",ccre_type_pELS,"ccre_type_pELS,CTCF-bound",is_protein_coding,MAF_log
7636,AACAAAA,1,99267131,"0.992,0.007987",-0.055,0,ENSP00000359204.4,7.090493,0.007987,10,...,False,False,False,False,False,False,False,False,1,-4.829928
8915,AACAAAA,1,161294824,"0.9998,0.0001997",-1.399,0,0,7.090493,0.000200,10,...,False,False,False,False,False,False,False,False,1,-8.518194
908,AACAAAA,1,169903374,"0.9996,0.0003994",-1.708,0,0,7.090493,0.000399,10,...,False,False,False,False,False,False,False,False,1,-7.825297
5248,AACAAAA,1,82064640,"0.9998,0.0001997",-0.639,0,0,7.090493,0.000200,10,...,False,False,False,False,False,False,False,False,1,-8.518194
4127,AACAAAA,1,153830104,"0.9996,0.0003994",-1.016,0,ENSP00000490635.1,7.090493,0.000399,10,...,False,False,False,False,False,False,False,False,1,-7.825297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582302,TTCTTTT,1,156911602,"0.9792,0.02077",-0.868,0,0,6.576470,0.020770,10,...,False,False,False,False,False,False,False,False,1,-3.874241
7578604,TTCTTTT,1,26226773,"0.5325,0.4675",-1.496,0,0,6.576470,0.467500,10,...,False,False,False,False,False,False,False,False,1,-0.760356
7591827,TTCTTTT,1,229773166,"0.9998,0.0001997",0.243,0,0,6.576470,0.000200,10,...,False,False,False,False,False,False,False,False,1,-8.518194
7579876,TTCTTTT,1,106091814,"0.7051,0.2949",-1.192,0,0,6.576470,0.294900,10,...,False,False,False,False,False,False,False,False,1,-1.221119


In [16]:
print("\nColumn data types:")
print(sampled_df_100_transformed.dtypes)  # Display the data types of the columns


Column data types:
sequence_context                                 object
seqnames                                          int64
start                                             int64
CAF                                              object
PhyloP_score                                    float64
is_CpG                                            int64
protein_id                                       object
experimental_ln_FI                              float64
MAF                                             float64
label_counts                                      int64
gene_type_TEC                                      bool
gene_type_lncRNA                                   bool
gene_type_miRNA                                    bool
gene_type_misc_RNA                                 bool
gene_type_polymorphic_pseudogene                   bool
gene_type_processed_pseudogene                     bool
gene_type_protein_coding                           bool
gene_type_rRNA_pseudogene   

Standardization (Z-score Normalization) of TDG values is not needed in XGBoost

In [17]:
# Remove MAF and CAF
sampled_df_100_transformed = sampled_df_100_transformed.drop(columns=['MAF', 'CAF', 'protein_id'])

In [18]:
sampled_df_100_transformed

,sequence_context,seqnames,start,PhyloP_score,is_CpG,experimental_ln_FI,label_counts,gene_type_TEC,gene_type_lncRNA,gene_type_miRNA,...,ccre_type_DNase-H3K4me3,"ccre_type_DNase-H3K4me3,CTCF-bound",ccre_type_PLS,"ccre_type_PLS,CTCF-bound",ccre_type_dELS,"ccre_type_dELS,CTCF-bound",ccre_type_pELS,"ccre_type_pELS,CTCF-bound",is_protein_coding,MAF_log
7636,AACAAAA,1,99267131,-0.055,0,7.090493,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-4.829928
8915,AACAAAA,1,161294824,-1.399,0,7.090493,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-8.518194
908,AACAAAA,1,169903374,-1.708,0,7.090493,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-7.825297
5248,AACAAAA,1,82064640,-0.639,0,7.090493,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-8.518194
4127,AACAAAA,1,153830104,-1.016,0,7.090493,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-7.825297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582302,TTCTTTT,1,156911602,-0.868,0,6.576470,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-3.874241
7578604,TTCTTTT,1,26226773,-1.496,0,6.576470,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-0.760356
7591827,TTCTTTT,1,229773166,0.243,0,6.576470,10,False,False,False,...,False,False,False,False,False,False,False,False,1,-8.518194
7579876,TTCTTTT,1,106091814,-1.192,0,6.576470,10,False,True,False,...,False,False,False,False,False,False,False,False,1,-1.221119


In [19]:
# save the processed data:
sampled_df_100_transformed.to_csv('mutations_TDG_genomic_processed.csv', index=False)

In [20]:
# Define features:
feature_names = ['is_CpG','is_protein_coding', 'experimental_ln_FI']
#feature_indexes = list(range(7, 32))   # for one hot encoded

In [22]:
one_hot_feature_names = sampled_df_100_transformed.columns[7:31].tolist()
all_features = feature_names + one_hot_feature_names  # Combine named & indexed features
all_features

['is_CpG',
 'is_protein_coding',
 'experimental_ln_FI',
 'gene_type_TEC',
 'gene_type_lncRNA',
 'gene_type_miRNA',
 'gene_type_misc_RNA',
 'gene_type_polymorphic_pseudogene',
 'gene_type_processed_pseudogene',
 'gene_type_protein_coding',
 'gene_type_rRNA_pseudogene',
 'gene_type_snRNA',
 'gene_type_snoRNA',
 'gene_type_transcribed_processed_pseudogene',
 'gene_type_transcribed_unitary_pseudogene',
 'gene_type_transcribed_unprocessed_pseudogene',
 'gene_type_unitary_pseudogene',
 'gene_type_unprocessed_pseudogene',
 'ccre_type_CTCF-only,CTCF-bound',
 'ccre_type_DNase-H3K4me3',
 'ccre_type_DNase-H3K4me3,CTCF-bound',
 'ccre_type_PLS',
 'ccre_type_PLS,CTCF-bound',
 'ccre_type_dELS',
 'ccre_type_dELS,CTCF-bound',
 'ccre_type_pELS',
 'ccre_type_pELS,CTCF-bound']

In [23]:
feature_indexes = list(range(7, 32))   # for one hot encoded
sampled_df_100_transformed.iloc[:, feature_indexes] = sampled_df_100_transformed.iloc[:, feature_indexes].astype(int)
sampled_df_100_transformed

,sequence_context,seqnames,start,PhyloP_score,is_CpG,experimental_ln_FI,label_counts,gene_type_TEC,gene_type_lncRNA,gene_type_miRNA,...,ccre_type_DNase-H3K4me3,"ccre_type_DNase-H3K4me3,CTCF-bound",ccre_type_PLS,"ccre_type_PLS,CTCF-bound",ccre_type_dELS,"ccre_type_dELS,CTCF-bound",ccre_type_pELS,"ccre_type_pELS,CTCF-bound",is_protein_coding,MAF_log
7636,AACAAAA,1,99267131,-0.055,0,7.090493,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-4.829928
8915,AACAAAA,1,161294824,-1.399,0,7.090493,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-8.518194
908,AACAAAA,1,169903374,-1.708,0,7.090493,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-7.825297
5248,AACAAAA,1,82064640,-0.639,0,7.090493,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-8.518194
4127,AACAAAA,1,153830104,-1.016,0,7.090493,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-7.825297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582302,TTCTTTT,1,156911602,-0.868,0,6.576470,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-3.874241
7578604,TTCTTTT,1,26226773,-1.496,0,6.576470,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.760356
7591827,TTCTTTT,1,229773166,0.243,0,6.576470,10,0,0,0,...,0,0,0,0,0,0,0,0,1,-8.518194
7579876,TTCTTTT,1,106091814,-1.192,0,6.576470,10,0,1,0,...,0,0,0,0,0,0,0,0,1,-1.221119


In [24]:
X = sampled_df_100_transformed[all_features]
y = sampled_df_100_transformed['MAF_log']  # Replace with actual target column name
X, y

(         is_CpG  is_protein_coding  experimental_ln_FI  gene_type_TEC  \
 7636          0                  1            7.090493              0   
 8915          0                  1            7.090493              0   
 908           0                  1            7.090493              0   
 5248          0                  1            7.090493              0   
 4127          0                  1            7.090493              0   
 ...         ...                ...                 ...            ...   
 7582302       0                  1            6.576470              0   
 7578604       0                  1            6.576470              0   
 7591827       0                  1            6.576470              0   
 7579876       0                  1            6.576470              0   
 7581485       0                  1            6.576470              0   
 
          gene_type_lncRNA  gene_type_miRNA  gene_type_misc_RNA  \
 7636                    0                0

In [25]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score 

In [26]:
# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost regressor
model = XGBRegressor(tree_method='hist', 
                     enable_categorical=False, 
                     random_state=42)

In [ ]:
# Train the model
model.fit(X_train, y_train)